In [ ]:
##基本的に以下を実行していけば、result_rensyu.csvとして配布したものと同じものが出てくるはず

In [2]:
##MeCabと品詞分解用の辞書のインストール（呪文だと思えば良い）
# MeCabのインストール
!apt install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3

# mecab-ipadic-NEologd（すごい辞書）のインストール
!apt install git make curl xz-utils file
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

!ln -s /etc/mecabrc /usr/local/etc/mecabrc

##Mecabで使う辞書を上でインストールしたすごい辞書にするための準備
#辞書をインストールした場所（path）を探してもらう（Mecabにはそこにあるものを辞書にしろと後で命令する）

#pathを探すために必要なやつを用意する
import subprocess

#subprocessに探してもらうやつを教える
cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'

#pathを探してもらう
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
#pathをMeCabに分かるよう書き換えてあげて、MeCabにはdicdirにある辞書を使えと命令すればいいようにする
dicdir='-d {0}'.format(path)

##参考サイト
# https://qiita.com/hnishi/items/0d32a778e375a99aff13


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-jumandic mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
0 upgraded, 8 newly installed, 0 to remove and 49 not upgraded.
Need to get 29.0 MB of archives.
After this operation, 277 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab-dev amd64 0.996-5 [308 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-utils amd64 0.996-5 [4,856 B]
Get:4 http://archive.ubuntu.com/ub

In [3]:
##MeCabを使ってみる

#MeCabを呼んできて、使えるようにする
import MeCab

#MeCabにdicdirにある辞書を使って、「鬼滅の刃」を品詞分解（parse）しろと命じる
MeCab.Tagger(dicdir).parse("鬼滅の刃")

'鬼滅の刃\t名詞,固有名詞,一般,*,*,*,鬼滅の刃,キメツノヤイバ,キメツノヤイバ\nEOS\n'

In [4]:
##MeCabの出力は、上のように「単語（区切り文字）品詞・・・」というように出てくる
##だから、そこからやりたい分析に必要なものを取り出して、使いやすいように整理していく

In [5]:
##使うデータをエクセルの表みたいな形式（データフレーム）にして読み込む

#データフレームを扱うための強い味方（pandas）を呼び出す
import pandas as pd

#pandasを使って、データを読み込み、dfと名づける
df=pd.read_csv("koikekaiken_200407.csv")

In [6]:
#dfを確認
df

,日付,本文,URL
0,2020/4/7,【知事】本日の感染症が判明した方々のことについてお伝えいたします。本日は80名の感染が判明い...,https://www.metro.tokyo.lg.jp//tosei/governor/...


In [8]:
#dfの本文部分だけを抽出する
text_list=df["本文"].to_list()

text_list

['【知事】本日の感染症が判明した方々のことについてお伝えいたします。本日は80名の感染が判明いたしております。そして、4月1日からの一週間でありますけれども、都内の感染者数が4月1日に66名、そして97名、89名、117名、そして143名、83名に下がって、今日は80名でありますけれども、いずれにしましても大変高水準で推移しているところであります。累計致しますと1,195名となりまして、非常に逼迫した状況が続いている、重大局面は変わらずということでございます。本日、国が7つの都府県を対象といたしまして、令和2年4月7日から5月6日までの30日間を期間といたします緊急事態宣言を発したところであります。都におきましては全域がその対象区域となったわけでございます。国におきます、緊急事態の事態宣言を受けまして、都としてまずは、感染の拡大防止のためにも、重要な柱と致しましては、まず都民の皆様に対して、4月8日午前0時00分、先ほど申し上げた、特措法が発令される、そして特措法の第45条第1項に基づきまして、外出の自粛要請を行います。区域は東京都内全域、期間は先ほど申し上げたように5月6日まで。緊急事態措置の一環としてお示ししております施設の使用制限などでございますけれども、現在は対象施設について国との間で調整を行っているところでございます。引き続き、この具体的な内容につきましては、国との調整を行って、都といたしましては、現状を鑑みますと、できるだけ早く明確にしていきたいとのことから、4月9日までに都としての使用制限に対しての、成案を得たいと思います。そして、外出自粛の効果も踏まえながら、翌10日の発表、11日からの実施、こういうスケジュール感でいきたいと存じます。東京といたしましては、大変スピード感も重要ということでございまして、このような日程を考えたいと思います。ポイントでありますけれども、やはり引き続きになりますけれど、改めて申し上げたいと思います。大事なことなので、もう一度と言うことですが、ステイホーム、お家にいてください、このアナウンスであります。外出を控えて下さい。外出しないでください。そして、外出せざるを得ない場合には密閉、密集、密接、この3つの密を避ける。もうこれ何度も申し上げていることでございます。どうぞよろしくお願いいたします。そして人と人との間隔を約2m、

In [9]:
##形態素解析をしていく
##MeCabの出力例は読み込ませる文の一語ごとに「鬼滅の刃	名詞,固有名詞,一般,*,*,*,…」となる
##ごちゃついてるから、そこからやりたい分析に必要なものを取り出して、使いやすいように整理していく
##（ややこしいので呪文だと思ってください、たぶんもっと綺麗に書けるから恥ずかしい…）

tagger = MeCab.Tagger(dicdir)
result_list=[]

for text in text_list:
    result = tagger.parse(text)
    result=result.replace("EOS","")
    result_list.append(result)

hinshi_lists=[]
for result in result_list:
    hinshi_list=[]
    result1=result.split("\n")
    for word in result1:
        word1=word.split("\t")
        if word1[0]!="":
            hinshi=[word1[0]]+word1[1].split(",")
            hinshi_list.append(hinshi)
    hinshi_lists.append(hinshi_list)

hh=hinshi_lists

nhinshi_lists=[]
for hinshi_list in hinshi_lists:
    nhinshi_list=[]
    for hinshi in hinshi_list:
        if len(hinshi)!=10:
            hinshi.append("*")
            hinshi.append("*")
            nhinshi_list.append(hinshi)
        else:
            nhinshi_list.append(hinshi)
    nhinshi_lists.append(nhinshi_lists)

#これで一文ごとに分割された語のリストのリストができる
go_lists=[]
for hinshi_list in hinshi_lists:
    dd=pd.DataFrame(hinshi_list, columns=["MOTO","H1","H2","H3","H4","H5","H6","GO","YOMI1","YOMI2"])
    ddd=dd[dd["H1"].isin(['名詞', '動詞','形容詞','形容動詞'])]
    dddd=ddd[ddd["H2"].isin(["自立","サ変接続","ナイ形容詞語幹","一般","形容動詞語幹","固有名詞","副詞可能"])]
    ddddd=dddd.query("H3 !=['人名','組織','地名','地域'] ")
    go_list=ddddd["GO"].to_list()
    go_lists.append(go_list)


In [ ]:
##上で出力されたものを見てみる（名詞、動詞、形容詞、形容動詞の語の原形だけを取り出して並べた）
##これの語と、感情語辞書のマッチングを行っていく
go_lists

In [11]:
##感情語辞書をデータフレーム形式で読み込み、dfjisyoと名前をつける
#読み込み
dfjisyo=pd.read_csv("道徳辞書.csv")

#試しに見てみる
dfjisyo


,HarmVirtue,HarmVice,FairnessVirtue,FairnessVice,IngroupVirtue,IngroupVice,AuthorityVirtue,AuthorityVice,PurityVirtue,PurityVice,MoralityGeneral
0,守る,損害,フェア,不当,共に,奔放,従う,挑戦,祈り,不貞,利益
1,安全,害,適正,不正,互いに,分離,遵奉,反抗,神父,むかつ,悪人
2,無事,有害,公正,不公,円満な,引き離,従順,侮り,高僧,嫌悪,悪党
3,無難,傷害,公平,卑怯,国,隔離,服従,傲慢な,教主,腐敗,悪漢
4,落ち着,危害,並ぶ,不穏当,国民,外国人,忠順,抵抗,牧師,堕落,悪用
...,...,...,...,...,...,...,...,...,...,...,...
117,NaN,NaN,NaN,NaN,NaN,NaN,君子,NaN,NaN,NaN,NaN
118,NaN,NaN,NaN,NaN,NaN,NaN,謙遜,NaN,NaN,NaN,NaN
119,NaN,NaN,NaN,NaN,NaN,NaN,礼儀,NaN,NaN,NaN,NaN
120,NaN,NaN,NaN,NaN,NaN,NaN,見せしめ,NaN,NaN,NaN,NaN


In [12]:
##マッチング作業がしやすいようにゴニョゴニョする
kanjol=list(dfjisyo.columns)
jd=dfjisyo.to_dict()

In [14]:
##ゴニョゴニョとマッチングをしていく
##最終的にddllという箱（名前適当）に、道徳感情ごとに語が出てくるたびに格納される

#マッチング
ddll=[]
for go_list in go_lists:
    
    #print(go_list)
    #ddl=[]
    dd={}
    for kanjo in kanjol:
        #print(kanjo)
        jdset=set(jd[kanjo].values())
        wl=[]
        
        for i in jdset: 
            #print(jdset)
            #print(i)
            
            for go in go_list:
                #print(go)
                #print(i)
               
                if str(i) ==str(go):
                    wl.append(go)
                else:
                    pass
            dd[kanjo]=wl
            #dd[kanjo]=cc
        #ddl.append(dd)
    ddll.append(dd)
  
#結果表示
ddll

[{'AuthorityVice': [],
  'AuthorityVirtue': ['資格',
   '義務',
   '定める',
   '定める',
   '法律',
   '法律',
   '法律',
   '指示',
   '指示',
   '規定',
   '規定',
   '規定',
   '体制',
   '従う',
   '従う',
   '運営'],
  'FairnessVice': ['優先', '不当', '差別'],
  'FairnessVirtue': [],
  'HarmVice': [],
  'HarmVirtue': ['保全',
   '保全',
   '安全',
   '安全',
   '守る',
   '守る',
   '守る',
   '守る',
   '守る',
   '守る',
   '守る',
   '守る'],
  'IngroupVice': [],
  'IngroupVirtue': ['協力',
   '協力',
   '協力',
   '協力',
   '協力',
   '協力',
   '協力',
   '協力',
   '協力',
   '協力',
   '協力',
   '協力',
   '協力',
   '社会',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '国',
   '共有',
   '共有',
   '共有',
   '共有',
   '家族'],
  'MoralityGeneral': ['利益', '利益', '利益', '利益', '利益', '利益'],
  'PurityVice': ['感染', '感染', '感染', '感染', '感染', '感染', '感染', '感染', '感染'],
  'PurityVirtue': ['抑制']}]

In [15]:
##上のままだと見にくいので、kanjo_dfとして、各感情の出てきた数をカウントした表を出力

#ゴニョゴニョやる
kdd={}
for n, ddl in enumerate(ddll): 
    kd={}
    for kanjo, wl in ddl.items():
        wll=len(wl)
        kd[kanjo]=wll
    kdd[n]=kd

kanjo_df=pd.DataFrame(kdd).T

#結果の確認
kanjo_df

,AuthorityVice,AuthorityVirtue,FairnessVice,FairnessVirtue,HarmVice,HarmVirtue,IngroupVice,IngroupVirtue,MoralityGeneral,PurityVice,PurityVirtue
0,0,16,3,0,0,12,0,44,6,9,1


In [18]:
##最初に読み込んだdfと結合する
#結合の呪文（concat）
result_df=pd.concat([df, kanjo_df], axis=1)

#確認
result_df

,日付,本文,URL,AuthorityVice,AuthorityVirtue,FairnessVice,FairnessVirtue,HarmVice,HarmVirtue,IngroupVice,IngroupVirtue,MoralityGeneral,PurityVice,PurityVirtue
0,2020/4/7,【知事】本日の感染症が判明した方々のことについてお伝えいたします。本日は80名の感染が判明い...,https://www.metro.tokyo.lg.jp//tosei/governor/...,0,16,3,0,0,12,0,44,6,9,1


In [19]:
##result_dfをcsvとして書き出し

result_df.to_csv("rensyu_result.csv")

##ファイルのところに追加されるから、あとは自由に自分のPCにダウンロードできる